In [ ]:
%pylab inline

In [15]:
import pyccl as ccl
import sacc
import sys
import numpy as np
sys.path.append('../')
import TJPCov.tjpcov as tjpcov

In [3]:
ccl.__version__

'2.5.1'

In [4]:
cosmo = ccl.Cosmology(Omega_c = 0.27, Omega_b = 0.045, h = 0.67, sigma8 = 0.83, n_s = 0.96,transfer_function='boltzmann_class')

In [32]:
twopoint_data = sacc.Sacc.load_fits('TJPCov/twopoint_data.sacc') #provided by Emily Longley
twopoint_data.get_data_types()
# Angular Cl is calculated later on

FileNotFoundError: [Errno 2] No such file or directory: 'TJPCov/twopoint_data.sacc'

In [14]:
twopoint_data.tracers
# Only has z and dndz in it I think? Perhaps don't need to put anything in SACC?

{'source_0': <sacc.tracers.NZTracer at 0x7fb3ca1bc0a0>,
 'source_1': <sacc.tracers.NZTracer at 0x7fb3ca1bc520>,
 'source_2': <sacc.tracers.NZTracer at 0x7fb3ca1bc790>,
 'source_3': <sacc.tracers.NZTracer at 0x7fb3ca1bcb50>,
 'lens_0': <sacc.tracers.NZTracer at 0x7fb3ca1bc250>}

In [7]:
twopoint_data.metadata['fsky']=0.3

In [10]:
d2r=np.pi/180

In [16]:
#FIXME: ell_bins and th_bins should be passed by sacc. ell0 and th can be decided based on binning or can also be passed by sacc.
twopoint_data.metadata['ell']=np.arange(2,500)
twopoint_data.metadata['ell_bins']=np.arange(2,500,20)
th_min=2.5/60 # in degrees
th_max=250./60
n_th_bins=20
twopoint_data.metadata['th_bins']=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)

th=np.logspace(np.log10(th_min*0.98),np.log10(1),n_th_bins*30) #covariance is oversampled at th values and then binned.
th2=np.linspace(1,th_max*1.02,n_th_bins*30) #binned covariance can be sensitive to the th values. Make sue you check convergence for your application
# th2=np.logspace(np.log10(1),np.log10(th_max),60*6)
twopoint_data.metadata['th']=np.unique(np.sort(np.append(th,th2)))
thb=0.5*(twopoint_data.metadata['th_bins'][1:]+twopoint_data.metadata['th_bins'][:-1])

In [17]:
#The spin based factor to decide the wigner transform. Based on spin of tracers. Sometimes we may use s1_s2 to denote these factors
WT_factors={}
WT_factors['lens','source']=(0,2)
WT_factors['source','lens']=(2,0) #same as (0,2)
WT_factors['source','source']={'plus':(2,2),'minus':(2,-2)}
WT_factors['lens','lens']=(0,0)

In [18]:
#### Wigner Transform setup... 
WT_kwargs={'l': twopoint_data.metadata['ell'],'theta': twopoint_data.metadata['th']*d2r,'s1_s2':[(2,2),(2,-2),(0,2),(2,0),(0,0)]}
%time WT=tjpcov.wigner_transform(**WT_kwargs)

CPU times: user 655 ms, sys: 2.08 s, total: 2.74 s
Wall time: 2.96 s


In [28]:
tracer_combs = twopoint_data.get_tracer_combinations()

In [31]:
tracer_combs

[('lens_0', 'lens_0'),
 ('source_0', 'lens_0'),
 ('source_1', 'lens_0'),
 ('source_2', 'lens_0'),
 ('source_3', 'lens_0'),
 ('source_0', 'source_0'),
 ('source_1', 'source_0'),
 ('source_1', 'source_1'),
 ('source_2', 'source_0'),
 ('source_2', 'source_1'),
 ('source_2', 'source_2'),
 ('source_3', 'source_0'),
 ('source_3', 'source_1'),
 ('source_3', 'source_2'),
 ('source_3', 'source_3')]

In [20]:
twopoint_data.get_data_types()

['galaxy_density_xi',
 'galaxy_shearDensity_xi_t',
 'galaxy_shear_xi_minus',
 'galaxy_shear_xi_plus']

In [21]:
twopoint_data.tracers

{'source_0': <sacc.tracers.NZTracer at 0x7fb3ca1bc0a0>,
 'source_1': <sacc.tracers.NZTracer at 0x7fb3ca1bc520>,
 'source_2': <sacc.tracers.NZTracer at 0x7fb3ca1bc790>,
 'source_3': <sacc.tracers.NZTracer at 0x7fb3ca1bcb50>,
 'lens_0': <sacc.tracers.NZTracer at 0x7fb3ca1bc250>}

In [22]:
#this function will generate and return CCL_tracer objects and also compute the noise for all the tracers
def get_tracer_info(two_point_data={}):
    ccl_tracers={}
    tracer_Noise={}
    for tracer in twopoint_data.tracers:
        tracer_dat=twopoint_data.get_tracer(tracer)
        z= tracer_dat.z
        #FIXME: Following should be read from sacc dataset.
        Ngal = 26. #arc_min^2
        sigma_e=.26 #shape noise per component.
        b = 1.5*np.ones(len(z)) #Galaxy bias (constant with scale and z)
        AI = .5*np.ones(len(z)) #Galaxy bias (constant with scale and z)
        Ngal=Ngal*3600/d2r**2
        #red_frac=0.2*np.ones(len(z))
        
        dNdz = tracer_dat.nz
        dNdz/=(dNdz*np.gradient(z)).sum()
        dNdz*=Ngal
        
        if 'source' in tracer:  
            ccl_tracers[tracer]=ccl.WeakLensingTracer(cosmo, dndz=(z, dNdz),ia_bias=(z,AI)) #CCL automatically normalizes dNdz
            tracer_Noise[tracer]=sigma_e**2/Ngal
        elif 'lens' in tracer:
            tracer_Noise[tracer]=1./Ngal
            ccl_tracers[tracer]=ccl.NumberCountsTracer(cosmo, has_rsd=False, dndz=(z,dNdz), bias=(z,b))
    return ccl_tracers,tracer_Noise

In [23]:
def get_cov_WT_spin(tracer_comb=None):
    tracers=tuple(i.split('_')[0] for i in tracer_comb)
    return WT_factors[tracers]

In [24]:
#compute a single covariance matrix for a given pair of C_ell or xi.  
def cl_gaussian_cov(tracer_comb1=None,tracer_comb2=None,ccl_tracers=None,tracer_Noise=None,two_point_data=None,do_xi=False,
                    xi_plus_minus1='plus',xi_plus_minus2='plus'):  
    #fsky should be read from the sacc
    #tracers 1,2,3,4=tracer_comb1[0],tracer_comb1[1],tracer_comb2[0],tracer_comb2[1]
    ell=two_point_data.metadata['ell']
    cl={}
    cl[13] = ccl.angular_cl(cosmo, ccl_tracers[tracer_comb1[0]], ccl_tracers[tracer_comb2[0]], ell)
    cl[24] = ccl.angular_cl(cosmo, ccl_tracers[tracer_comb1[1]], ccl_tracers[tracer_comb2[1]], ell)
    cl[14] = ccl.angular_cl(cosmo, ccl_tracers[tracer_comb1[0]], ccl_tracers[tracer_comb2[1]], ell)
    cl[23] = ccl.angular_cl(cosmo, ccl_tracers[tracer_comb1[1]], ccl_tracers[tracer_comb2[0]], ell)
    
    SN={}
    SN[13]=tracer_Noise[tracer_comb1[0]] if tracer_comb1[0]==tracer_comb2[0]  else 0
    SN[24]=tracer_Noise[tracer_comb1[1]] if tracer_comb1[1]==tracer_comb2[1]  else 0
    SN[14]=tracer_Noise[tracer_comb1[0]] if tracer_comb1[0]==tracer_comb2[1]  else 0
    SN[23]=tracer_Noise[tracer_comb1[1]] if tracer_comb1[1]==tracer_comb2[0]  else 0
    
    if do_xi:
        norm=np.pi*4*two_point_data.metadata['fsky']
    else: #do c_ell
        norm=(2*ell+1)*np.gradient(ell)*two_point_data.metadata['fsky']

    coupling_mat={}
    coupling_mat[1324]=np.eye(len(ell)) #placeholder
    coupling_mat[1423]=np.eye(len(ell)) #placeholder
    
    cov={}
    cov[1324]=np.outer(cl[13]+SN[13],cl[24]+SN[24])*coupling_mat[1324]
    cov[1423]=np.outer(cl[14]+SN[14],cl[23]+SN[23])*coupling_mat[1423]
    
    if self.do_xi and np.all(np.array(tracers)=='shear'): #this add the B-mode shape noise contribution. We assume B-mode power (C_ell) is 0
        Bmode_F=1
        if xi_plus_minus1!=xi_plus_minus2:
            Bmode_F=-1 #in the cross term, this contribution is subtracted. eq. 29-31 of https://arxiv.org/pdf/0708.0387.pdf
        cov[1324]+=np.outer(cl[13]*0+SN[13],cl[24]*0+SN[24])*coupling_mat[1324]*Bmode_F
        cov[1423]+=np.outer(cl[14]*0+SN[14],cl[23]*0+SN[23])*coupling_mat[1423]*Bmode_F
        
    cov['final']=cov[1423]+cov[1324]
    
    if do_xi:
        s1_s2_1=get_cov_WT_spin(tracer_comb=tracer_comb1)
        s1_s2_2=get_cov_WT_spin(tracer_comb=tracer_comb2)
        
        if isinstance(s1_s2_1,dict):
            s1_s2_1=s1_s2_1[xi_plus_minus1] 
        if isinstance(s1_s2_2,dict):
            s1_s2_2=s1_s2_2[xi_plus_minus2]
            
        th,cov['final']=WT.projected_covariance2(l_cl=ell,s1_s2=s1_s2_1, s1_s2_cross=s1_s2_2,
                                                      cl_cov=cov['final'])

    cov['final']/=norm
    
    if do_xi:
        thb,cov['final_b']=tjpcov.bin_cov(r=th/d2r,r_bins=two_point_data.metadata['th_bins'],cov=cov['final']) 
    else:
        if two_point_data.metadata['ell_bins'] is not None:
            lb,cov['final_b']=tjpcov.bin_cov(r=ell,r_bins=two_point_data.metadata['ell_bins'],cov=cov['final']) 
            
#     cov[1324]=None #if want to save memory
#     cov[1423]=None #if want to save memory
    return cov

In [25]:
#compute all the covariances and then combine them into one single giant matrix
def get_all_cov(two_point_data={},do_xi=False):
    #FIXME: Only input needed should be two_point_data, which is the sacc data file. Other parameters should be included within sacc and read from there.
    ccl_tracers,tracer_Noise=get_tracer_info(two_point_data=two_point_data)
    tracer_combs=two_point_data.get_tracer_combinations()# we will loop over all these
    N2pt=len(tracer_combs)
    if two_point_data.metadata['ell_bins'] is not None:
        Nell_bins=len(two_point_data.metadata['ell_bins'])-1
    else:
        Nell_bins=len(two_point_data.metadata['ell'])
    if do_xi:
        Nell_bins=len(two_point_data.metadata['th_bins'])-1
    cov_full=np.zeros((Nell_bins*N2pt,Nell_bins*N2pt))
    for i in np.arange(N2pt):
        print("{}/{}".format(i+1, N2pt))
        tracer_comb1=tracer_combs[i]
        indx_i=i*Nell_bins
        for j in np.arange(i,N2pt):
            tracer_comb2=tracer_combs[j]
            indx_j=j*Nell_bins
            cov_ij=cl_gaussian_cov(tracer_comb1=tracer_comb1,tracer_comb2=tracer_comb2,ccl_tracers=ccl_tracers,
                                        tracer_Noise=tracer_Noise,do_xi=do_xi,two_point_data=two_point_data)
            if do_xi or two_point_data.metadata['ell_bins'] is not None:
                cov_ij=cov_ij['final_b']
            else:
                cov_ij=cov_ij['final']
            cov_full[indx_i:indx_i+Nell_bins,indx_j:indx_j+Nell_bins]=cov_ij
            cov_full[indx_j:indx_j+Nell_bins,indx_i:indx_i+Nell_bins]=cov_ij.T
    return cov_full

In [26]:
#C_ell covariance
cov_cl=get_all_cov(two_point_data=twopoint_data,do_xi=False)
#xi covariance .... right now shear-shear is xi+ only. xi- needs to be added in the loops.
cov_xi=get_all_cov(two_point_data=twopoint_data,do_xi=True)

1/15


ModuleNotFoundError: No module named 'classy'

In [ ]:
err=np.sqrt(np.diag(cov_cl))
corr_m=cov_cl/np.outer(err,err)
pcolor(corr_m,vmin=-1,vmax=1,cmap='seismic')

In [ ]:
err=np.sqrt(np.diag(cov_xi))
corr_m=cov_xi/np.outer(err,err)
pcolor(corr_m,vmin=-1,vmax=1,cmap='seismic')